In [ ]:
import os

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import networkx as nx

import matplotlib.pyplot as plt



%config Completer.use_jedi = False

In [ ]:
# definitions 

inpath_skorea = "../data/coronavirusdataset/"

plots_path = "../plots/"
if not os.path.exists(plots_path): os.mkdir(plots_path)

In [ ]:
# read data
#path = 'kaggle/input/coronavirusdataset/'
path = inpath_skorea

case = pd.read_csv(path+'Case.csv')
time = pd.read_csv(path+'Time.csv')
t_age = pd.read_csv(path+'TimeAge.csv')
t_gender = pd.read_csv(path+'TimeGender.csv')
t_provin = pd.read_csv(path+'TimeProvince.csv')
region = pd.read_csv(path+'Region.csv')
weather = pd.read_csv(path+'Weather.csv')
search = pd.read_csv(path+'SearchTrend.csv')
floating = pd.read_csv(path+'SeoulFloating.csv')
policy = pd.read_csv(path+'Policy.csv')

# further plots
- waterfall: 
    - patients
    - ... with tracking
    -

In [ ]:
owid = pd.read_csv('../data/owid-covid-data.csv')
newcases_sk = owid.loc[owid.location == 'South Korea', ['date', 'new_cases']]
newcases_sk['date'] = pd.to_datetime(newcases_sk.date)
newcases_sk.reset_index(inplace=True, drop=True)
newcases_sk

In [ ]:
case

In [ ]:
case.sort_values(by='confirmed', axis=0, ascending=False).head(20)

In [ ]:
from datetime import datetime

p_info = pd.read_csv(path+'PatientInfo.csv')
p_info.confirmed_date = pd.to_datetime(p_info.confirmed_date)
print(f'dropping {p_info.confirmed_date.isna().sum()} rows (missing confirmed date)')
p_info = p_info[~p_info.confirmed_date.isna()]

print(f'dropping {p_info.infected_by.isna().sum()} rows (missing confirmed date)')
p_info = p_info[~p_info.infected_by.isna()]

p_info.patient_id = p_info.patient_id.astype(int)

f = [len(s) == 10 for s in p_info.infected_by]
print(f'dropping {p_info.shape[0] - sum(f)} rows (infected by value invalid)')
p_info = p_info.loc[f,:]   
p_info.infected_by = p_info.infected_by.astype(int)

date_after_100_cases = datetime(2020, 2, 20)
p_info['days_since_start'] = (p_info.confirmed_date - date_after_100_cases).dt.days
p_info.days_since_start.isna().sum()
p_info.days_since_start

# p_info = p_info.loc[p_info.days_since_start >=0, ]

p_info


In [ ]:
p_info.days_since_start.max()

In [ ]:
p_info.info()

In [ ]:
p_info[~p_info.infected_by.isna()].infection_case.value_counts()

In [ ]:
p_info.infection_case.value_counts()
#p_info[p_info.infection_case == '1200001']

In [ ]:
p_info[p_info.infected_by == '2000000205']

In [ ]:
p_info.state.value_counts()

In [ ]:
def get_edges_from_p_info(p_info): 
    edges = pd.DataFrame({
        'source': p_info.infected_by, 
        'target': p_info.patient_id, 
        'days_since_start': p_info.days_since_start, 
        'contact_number': p_info.contact_number
    })
    

    edges = edges[~edges.source.isna()]
    edges.reset_index(drop=True, inplace=True)


    # filter invalid source ids (3)
    len('2000000205')
    # set([i if len(s) != 10 for i, s in enumerate(edgelist.source)])
    #f = [len(s) == 10 for s in edges.source]
    #edges = edges.loc[f,:]

    # cast source to int
    #edges.source = edges.source.astype(int)
    return edges


def describe_graph(G):   
    print(f'Size: {G.size()} | No nodes: {G.number_of_nodes()} | No edges: {G.number_of_edges()}')
   


In [ ]:
"""
https://networkx.org/documentation/stable/reference/generated/networkx.convert_matrix.from_pandas_edgelist.html
"""

if True:
    edges = get_edges_from_p_info(p_info)
else: 
    edges = pd.DataFrame(
        {
            "source": [0, 1, 2],
            "target": [2, 2, 3]
            ,
            "weight": [3, 4, 5],
            "color": ["red", "blue", "blue"],
        }
    )
    

# overview graph types
# https://networkx.org/documentation/stable/reference/classes/index.html
G = nx.from_pandas_edgelist(edges, edge_attr=True, create_using=nx.Graph)

 
describe_graph(G)


In [ ]:
edges.contact_number.isna().sum()


In [ ]:


# https://networkx.org/documentation/stable//reference/drawing.html#module-networkx.drawing.layout
#https://stackoverflow.com/questions/52400380/assign-edge-weights-to-a-networkx-graph-using-pandas-dataframe
#pos = nx.kamada_kawai_layout(G)
pos = nx.spring_layout(G, seed=444)



In [ ]:
from plotly.colors import hex_to_rgb
f"rgb{hex_to_rgb('9164A2')}"

In [ ]:
import matplotlib.font_manager
fonts_av = matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')
[f for f in fonts_av if 'Open' in f]

In [ ]:
font_color = 'white'
line_color = 'white'
 
    

marker_min = 1.
marker_max = 20.
height=500
font=dict(family="Open Sans", size=7.5, color=font_color)

new_template = dict(
    layout=go.Layout(
        title_font=dict(
        family="Open Sans", # choose a family
        size=16, # choose a size
        color=font_color # choose a color
    ), 
    legend_font=dict(
        family="Open Sans", # choose a family
        size=8, # choose a size
        color=font_color # choose a color
    ), 
    xaxis=dict(
        showline=True,
        linecolor=line_color,
        showgrid=False,
        title_font = dict(
            family="Open Sans", # choose a family
            size=10, # choose a size
            color=font_color # choose a color
        ), 
        tickfont=dict(family="Open Sans", # choose a family
            size=8, # choose a size
            color=font_color # choose a color
    )), 
    yaxis=dict(
        showline=True, 
        linecolor=line_color,
        showgrid=False, 
        title_font = dict(
            family="Open Sans", # choose a family
            size=10, # choose a size
            color=font_color # choose a color
        ), 
        tickfont=dict(family="Open Sans", # choose a family
            size=8, # choose a size
            color=font_color # choose a color
                     )),
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
    ))


def prepare_edge_trace(G, pos): 

    Xed=[]
    Yed=[]
    for edge in G.edges:
        Xed+=[pos[edge[0]][0],pos[edge[1]][0], None]
        Yed+=[pos[edge[0]][1],pos[edge[1]][1], None]
        
    return Xed, Yed


def prepare_node_trace(G, pos): 
    labels = list(pos.keys())

    Xv=[p[0] for p in pos.values()]
    Yv=[p[1] for p in pos.values()]
    
    return Xv, Yv, labels


def add_target_node_dimension(labels, edges, dim = 'days_since_start'):
    days_dim = []
    for l in labels: 
        days = edges.loc[edges.target  == l,:][dim].values
        if len(days) == 1: 
            days_dim.append((days[0]))
        elif len(days) > 1: 
            raise Exception('Error: len > 1')
        elif len(days) == 0:
            day_imp = edges[edges.source == l].days_since_start.min() - 1
            days_dim.append((day_imp))
        else: 
            raise Exception()
    return days_dim
            
            
def add_node_adjacencies(G): 
    
    node_adjacencies = []
    node_text = []
    for node, adjacencies in enumerate(G.adjacency()):
        node_adjacencies.append(len(adjacencies[1]))
        node_text.append('# of connections: '+str(len(adjacencies[1])))
    
    return node_adjacencies, node_text


def build_layout_networkgraph(height, width): 
    axis=dict(showline=False, # hide axis line, grid, ticklabels and  title
              zeroline=False,
              showgrid=False,
              showticklabels=False,
              title=''
              )

    width=width
    height=height
    
    opt = 1
    if opt == 1: 
        paper_bgcolor='rgba(0,0,0,0)'
        plot_bgcolor='rgba(0,0,0,0)'
    elif opt == 2: 
        paper_bgcolor=f"rgb{hex_to_rgb('2D3241')}" #'373C4B'#'rgba(0,0,0,0)',
        plot_bgcolor=f"rgb{hex_to_rgb('373C4B')}"#'red',
        
    layout=go.Layout(
        title=dict(
            text='Personal infection transmission patterns', 
            y=0.8,
            x=0.5,
            xanchor='center',
            yanchor='top'
            #x=0.5

        ),
        #font= dict(size=12),
        showlegend=False,
        autosize=False,
        width=width,
        height=height,
        paper_bgcolor=paper_bgcolor,
        plot_bgcolor=plot_bgcolor,
        xaxis=go.layout.XAxis(axis),
        yaxis=go.layout.YAxis(axis),
        margin=go.layout.Margin(
            l=40,
            r=40,
            b=85,
            t=100,
        ),
        hovermode='closest',
        annotations=[
               dict(
               showarrow=False,
                text='',
                xref='paper',
                yref='paper',
                x=0,
                y=-0.1,
                xanchor='left',
                yanchor='bottom',
                font=dict(
                size=14
                )
                )
            ]
        )
    return layout


def generateDiscreteColourScale(colour_set):
    #colour set is a list of lists
    colour_output = []
    num_colours = len(colour_set)
    divisions = 1./num_colours
    c_index = 0.
    # Loop over the colour set
    for cset in colour_set:
        num_subs = len(cset)
        sub_divisions = divisions/num_subs
        # Loop over the sub colours in this set
        for subcset in cset:
            colour_output.append((c_index,subcset))
            colour_output.append((c_index + sub_divisions-
                .001,subcset))
            c_index = c_index + sub_divisions
    colour_output[-1]=(1,colour_output[-1][1])
    return colour_output


def discrete_colorscale_uneven(bvals, colors):
    """
    bvals - list of values bounding intervals/ranges of interest
    colors - list of rgb or hex colorcodes for values in [bvals[k], bvals[k+1]],0<=k < len(bvals)-1
    returns the plotly  discrete colorscale
    """
    if len(bvals) != len(colors)+1:
        raise ValueError('len(boundary values) should be equal to  len(colors)+1')
    bvals = sorted(bvals)     
    nvals = [(v-bvals[0])/(bvals[-1]-bvals[0]) for v in bvals]  #normalized values
    
    dcolorscale = [] #discrete colorscale
    for k in range(len(colors)):
        dcolorscale.extend([[nvals[k], colors[k]], [nvals[k+1], colors[k]]])
    return dcolorscale


def plot_network(G, pos, edges, mark_nodes:list=None, marker_min=marker_min, marker_max=marker_max, height=height, width=height, font=font):
    """
    https://plotly.com/python/v3/igraph-networkx-comparison/
    """
    

    print('min', p_info.confirmed_date.min().date())
    print('max', p_info.confirmed_date.max().date())
    print('date_after_100_cases: ', date_after_100_cases.date())
    auto_ctracing = datetime(2020, 3, 26)
    print('auto_ctracing: ', auto_ctracing.date())
    auto_ctracing_day_after_day_100 = (auto_ctracing.date() - date_after_100_cases.date()).days
    print('auto_ctracing_day_after_day_100: ', auto_ctracing_day_after_day_100)



    # ------------------------------------edge_trace------------------------------------
    Xed, Yed = prepare_edge_trace(G, pos)
    edge_trace = go.Scatter(x=Xed,
                   y=Yed,
                   mode='lines',
                   line=dict(color='rgb(210,210,210)', width=1),
                   hoverinfo='none'
                   )

    # ------------------------------------node_trace------------------------------------

    """
    https://plotly.com/python/network-graphs/
    https://plotly.com/python/bubble-charts/#scaling-the-size-of-bubble-charts
    """
    
    Xv, Yv, labels = prepare_node_trace(G, pos)
    node_adjacencies, node_text = add_node_adjacencies(G)
    days_dim = add_target_node_dimension(labels, edges, dim = 'days_since_start')
    
    if mark_nodes == None: 
        size = node_adjacencies
    elif isinstance(mark_nodes, list): 
        size = [l in mark_nodes for l in labels]
    else: 
        raise Exception

    # red: CE4A50, yellow: F7C232, tuquise: 50BFC3, steel blue: 4D8FC4, pruple: 9164A2, feilchen: 9DADEA, orange: EF9336
    # orRed = EB652A
    opt = 6
    if opt == 1: 
        colorscale = [[0, '#F7C232'],[1, "rgb(145, 100, 162)"] ]
    elif opt == 2: 
        colorscale = [[0, f"rgb{hex_to_rgb('9164A2')}"], [1, f"rgb{hex_to_rgb('50BFC3')}"]]
    elif opt == 2.1: 
        colorscale = [['0', f"rgb{hex_to_rgb('FFE445')}"], ['1', f"rgb{hex_to_rgb('50BFC3')}"]]
    elif opt == 3: 
        colorscale = [[0, f"rgb{hex_to_rgb('9164A2')}"], [.5, f"rgb{hex_to_rgb('F7C232')}"], [1, 'white']]
    elif opt == 4: 
        colorscale = [[0, 'white'], [1, f"rgb{hex_to_rgb('50BFC3')}"]]
    elif opt == 5: 
        colorscale = generateDiscreteColourScale([['#FFE445'], ['#50BFC3']])
    elif opt == 6: 
        bvals = [min(days_dim),auto_ctracing_day_after_day_100, max(days_dim)]
        colors = ["rgb(145, 100, 162)", "rgb(243,203,70)"]
        colorscale = discrete_colorscale_uneven(bvals, colors)
        
    opt = 1
    if opt == 1:
        color = days_dim
        cbar_len = 1
        tickvals=None #[.25, .75], 
        ticktext=None#['0', '1']),
    elif opt == 2: 
        color = [0 if d > np.median(days_dim) else 1 for d in days_dim]
        print(pd.Series(color).value_counts())
        cbar_len = .25
        tickvals=[.25, .75]
        ticktext=['0', '1']
    elif opt == 3:
        color = days_dim
        cbar_len = 1
        bvals = np.array(bvals)
        tickvals = [np.mean(bvals[k:k+2]) for k in range(len(bvals)-1)] #position with respect to bvals where ticktext is displayed
        ticktext = ['s', 'q']#[f'<{bvals[1]}'] + [f'{bvals[k]}-{bvals[k+1]}' for k in range(1, len(bvals)-2)]+[f'>{bvals[-2]}']
    elif opt == 4: 
        color = days_dim
        cbar_len = 1
        bvals = np.array(bvals)
        tickvals = [np.mean(bvals[k:k+2]) for k in range(len(bvals)-1)] #position with respect to bvals where ticktext is displayed
        ticktext = ['s', 'q']


        
    node_trace=go.Scatter(
        x=Xv,
        y=Yv,
        mode='markers',
        name='net',
        marker=dict(
            showscale=True,
            colorscale=colorscale,
            reversescale=False,
            symbol='circle-dot',
            size=size,
            sizemode='area',
            sizeref=2.*max(node_adjacencies)/(marker_max**2), # 2. * max(array of size values) / (desired maximum marker size ** 2)
            sizemin=marker_min, # 
            color=color,
            opacity=1,
            #colorbar: https://plotly.com/python/reference/scatter/#scatter-marker-colorbar-outlinecolor
            colorbar=dict(
                thickness=7,
                len=cbar_len,
                title=dict(
                    text='', #Days since 100 cases
                    font=font
                ),
                xanchor='left',
                yanchor='bottom',
                y=.15,
                ypad=100,
                titleside='right', 
                tickvals=tickvals, #[.25, .75], 
                ticktext=ticktext, #['0', '1']),
                tickfont=font
            ),
            line=dict(color='rgb(50,50,50)', width=0.5)
        ),
        text=labels, 
        hoverinfo='text'
    )



    #node_trace.marker.color = days_dim
    #node_trace.marker.size = node_adjacencies
    #node_trace.text = node_text

    # ------------------------------------build figure------------------------------------

    annot=""

    data1=[edge_trace, node_trace]
    fig1=go.Figure(data=data1, layout=build_layout_networkgraph(height, width))
    fig1['layout']['annotations'][0]['text']=annot
    
    fig1.update_layout(template=new_template,height=height, width=height) 
    
    anno = dict(
        text='bb', 
        font= dict(size=20, color='red'), 
        yref='paper',
        y=1.06
        )

    # fig1.add_annotation(anno)

    return fig1



def plot_marker_legend(G, marker_min=marker_min, marker_max=marker_max, font=font, height=height): 
    node_adjacencies, node_text = add_node_adjacencies(G)

    #legend_list = np.arange(5,55,5)
    #y_list = np.linspace(0, 5, 4)
    
    x_vals = [1, 1, 1, 1]
    y_vals = [height/500,.125, .2, .3]
    size = [0, 5, 25, 50]

    fig = go.Figure()


    fig.add_trace(
        go.Scatter(
            x=x_vals, 
            y=y_vals, 
            mode='markers',               
            text=[1,2,3,4],
            textposition='middle right',
            textfont=dict(color='#E58606'),
            #mode='markers+text',
            marker=dict(
                size = size,
                color = "#d9d9d9",
                showscale = False, 
                #symbol='circle-dot',
                sizemode='area',
                sizeref=2.*max(node_adjacencies)/(marker_max**2), # 2. * max(array of size values) / (desired maximum marker size ** 2)
                sizemin=marker_min, 
                opacity=1
            )
        )
    )

    x_ref=1.02
    x_vals=[1.2,x_ref, x_ref, x_ref]
    for i in range(len(size)): 
        s = size[i]
        if s == 0: 
            s=''
        fig.add_annotation(
            x=1.2, 
            y=y_vals[i],
            align='left',
            xanchor='left',
            text=s,
            showarrow=False, 
            font=font
        )
        
    fig.add_annotation(
        align='left',
        x=.85, 
        xref='x',
        xanchor='left',
        y=.36,
        text='Number of direct infections',
        showarrow=False, 
        font=font
    )

    fig.add_annotation(
        align='right',
        x=-0.01, 
        xref='paper',
        xanchor='right',
        y=.74,
        text='Auto <br>contact <br>tracing',
        showarrow=False, 
        font=font
    )
    
    fig.add_annotation(
        align='left',
        x=.85, 
        xref='x',
        xanchor='left',
        y=.86,
        text='Days since 100 cases',
        showarrow=False, 
        font=font
    )

    #fig.layout = layout

    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False)

    fig.update_layout(template=new_template, height=height, width=height/(2.55*height/500))
    
    return fig



In [ ]:

marker_min = 1.
marker_max = 20.
height=500
font=dict(family='Open Sans', size=7.5, color=font_color)
#s

fig = plot_network(G, pos, edges, height=height, font=font,marker_min=marker_min, marker_max=marker_max)
fig.show()
#fig.write_image(os.path.join(plots_path, "infection_network.svg"))
fig.write_image(os.path.join(plots_path, "infection_network.png"), scale=5) # height=height, width=height
fig.write_image(os.path.join(plots_path, "infection_network_small.png"), scale=1)

marker_legend = plot_marker_legend(G, height=height, font=font,marker_min=marker_min, marker_max=marker_max)
marker_legend.show()
#marker_legend.write_image(os.path.join(plots_path, "marker_legend.svg"))
marker_legend.write_image(os.path.join(plots_path, "marker_legend.png"), scale=5)
marker_legend.write_image(os.path.join(plots_path, "marker_legend_small.png"), scale=1)

#py.iplot(fig1, filename='Coautorship-network-nx')


In [ ]:
N = 1000
t = np.linspace(0, 10, 100)
y = np.sin(t)

fig = go.Figure(data=go.Scatter(x=t, y=y, mode='markers'))

fig.show()

## largest compontents

https://networkx.org/documentation/stable//reference/algorithms/generated/networkx.algorithms.components.connected_components.html


In [ ]:
comps = {len(c):c for c in sorted(nx.connected_components(G), key=len, reverse=True)}
len(comps)
comps


S = [G.subgraph(c).copy() for c in nx.connected_components(G)]
S = [G.subgraph(c).copy() for c in sorted(nx.connected_components(G), key=len, reverse=True)]

print('n subgraphs:', len(S))

print('\nFirst:' )
S1 = S[0]
describe_graph(S1)




In [ ]:
def find_origin(S): 

    node_adjacencies = []
    for node, adjacencies in enumerate(S.adjacency()):
        node_adjacencies.append((adjacencies[0], len(adjacencies[1])))
    return sorted(node_adjacencies, key=lambda x: x[1], reverse=True)[0]



if True: 
    # extract p_info per subgraph
    comp_info = pd.DataFrame()
    for S1 in S: 
        tmp = p_info[
            (p_info.patient_id.isin(S1.nodes())) 
            | (p_info.infected_by.isin(S1.nodes()))
        ]
        #print(tmp)


        tmp_df = pd.DataFrame({
            'origin':find_origin(S1)[0],#tmp[tmp.confirmed_date == tmp.confirmed_date.min()].infected_by.mode()[0],
            'origin_size':find_origin(S1)[1],
            'size': S1.number_of_nodes(), 
            'min_date': tmp.confirmed_date.min(), 
            'days_since_start_mean': tmp.days_since_start.mean().round(), 
            'days_since_start_sd': tmp.days_since_start.std().round()
        }, index=[0])
    
        comp_info = pd.concat([comp_info, tmp_df])

comp_info.reset_index(inplace=True, drop=True)

comp_info['min_month'] = comp_info.min_date.dt.month

comp_info

In [ ]:
from plotly import express as px
px.violin(data_frame=comp_info, y='size', x = 'min_month')



In [ ]:
data=comp_info
time_col='min_date'
val_col='origin_size'
freq='W'

agg = data.set_index(time_col).groupby(pd.Grouper(freq=freq))[val_col].max().reset_index().rename(columns={0:val_col})

fig = px.line(agg, x=time_col, y=val_col, template='simple_white')

fig.show()
#fig.write_image(os.path.join(plots_path, 'p_info_cases_weekly.png'), scale=5)


In [ ]:
def plot_timeline(data, time_col, val_col, freq): 
    agg = data.set_index(time_col).groupby(pd.Grouper(freq=freq))[val_col].size().reset_index().rename(columns={0:val_col})

    fig = px.line(agg, x=time_col, y=val_col, template='simple_white')
    return fig

fig = plot_timeline(data=p_info, time_col='confirmed_date', val_col='infected_by', freq='W')
fig.show()
#fig.write_image(os.path.join(plots_path, 'p_info_cases_weekly.png'), scale=5)


In [ ]:
#fig, ax = plt.subplots(figsize=(10,10))
#nx.draw_networkx(S[0], pos=pos, ax=ax)

def plot_subgraph(S) -> plt.Figure:

    pos1 = nx.spring_layout(S, seed=444)

    fig = plot_network(S, pos1, edges, height=500, width=500)
    return fig

plot_subgraph(S[2]).show()

## find nodes realted to church 

In [ ]:
church_p_info = p_info[(p_info.city == 'Nam-gu') | (p_info.infection_case == 'Shincheonji Church')]
church_p_info
type(church_p_info.patient_id.unique().tolist())
church_nodes = list(set(church_p_info.patient_id.unique().tolist() + church_p_info.infected_by.unique().tolist()))
church_nodes

print(church_nodes)
church_p_info

In [ ]:
#[s if s.nodes.contains(1400000209) for s in S]
#filter(lambda s: 1400000209 in s.nodes, S)

found_S = [s for s in S if 1600000005 in s.nodes][0]

plot_subgraph(found_S).show()


In [ ]:
plot_network(G, pos, edges, mark_nodes=church_nodes)

In [ ]:
Smax = max(nx.connected_components(G), key=len)
len(Smax)

## over time ...

In [ ]:
import plotly.express as px
fig = px.histogram(p_info, x="days_since_start", template='simple_white')
print(f' data range: {p_info.confirmed_date.dt.date.min()}, {p_info.confirmed_date.dt.date.max()}')
fig.show()



In [ ]:
p_info[p_info.days_since_start > 100]

In [ ]:
#cases_per_day = p_info.groupby(p_info.confirmed_date.dt.weekofyear).size().reset_index().rename(columns={0:'cases'})
cases_per_day = p_info.set_index('confirmed_date').groupby(pd.Grouper(freq='W')).size().reset_index().rename(columns={0:'cases'})

fig = px.line(cases_per_day, x='confirmed_date', y='cases', template='simple_white')
fig.show()
fig.write_image(os.path.join(plots_path, 'p_info_cases_weekly.png'), scale=5)


In [ ]:
freq = 'W'

cases_per_day = p_info.set_index('confirmed_date').groupby(pd.Grouper(freq=freq)).size().reset_index().rename(columns={0:'cases'})

newcases = newcases_sk[(newcases_sk.date <= p_info.confirmed_date.max()) & (newcases_sk.date >= date_after_100_cases)]
newcases
total_cases_per_day = newcases.set_index('date').groupby(pd.Grouper(freq=freq)).new_cases.sum().reset_index().rename(columns={'new_cases':'total_cases'})
total_cases_per_day

case_coverage = cases_per_day.merge(total_cases_per_day, left_on='confirmed_date', right_on='date').drop(columns='date')
case_coverage['coverage'] = (case_coverage.cases / case_coverage.total_cases).round(2)
case_coverage

fig = px.area(case_coverage, x='confirmed_date', y='coverage', template='simple_white')
fig.update_yaxes(tickangle=-90)
fig.update_xaxes(autorange='reversed', visible = False)
fig.update_traces(fillcolor='red', line_color='red')
fig.update_layout(
    paper_bgcolor=f"rgb{hex_to_rgb('2D3241')}", #'373C4B'#'rgba(0,0,0,0)',
    plot_bgcolor=f"rgb{hex_to_rgb('373C4B')}"#'red',
)


fig.show()
fig.write_image(os.path.join(plots_path, 'case_coverage_weekly.png'), scale=5)



In [ ]:
p_info.set_index('confirmed_date').groupby(pd.Grouper(freq='W')).size()


In [ ]:
#G = nx.random_geometric_graph(200, 0.125)

for i, edge in enumerate(G.edges()): 
    print(i, edge)
    print(i,  G.nodes[edge[0]])
    print(i,  G.nodes[edge[1]])

    if i > 10: 
        break 


# Clustering

In [ ]:
if false: 
    # https://stackoverflow.com/questions/62902871/how-can-i-cluster-a-graph-g-created-in-networkx
    # https://towardsdatascience.com/node2vec-embeddings-for-graph-data-32a866340fef
    # https://github.com/eliorc/Medium/blob/master/Nod2Vec-FIFA17-Example.ipynb

    """
    Step 1: get the embedding of each node in the graph. 
    That means you need to get a continuous vector representation for each node. 
    You can use graph embedding methods like node2vec, deepwalk, etc to obtain the embedding. 
    Note that such methods preserve the structural similarity between the nodes of a graph in the vector representation 
    (embedding space). The following example shows how you can do that.
    """

    import networkx as nx
    G=nx.Graph();
    G=nx.read_edgelist("edges.txt") # edges.txt contains the edge list of your graph

    # help to draw https://networkx.github.io/documentation/networkx-1.9/examples/drawing/labels_and_colors.html
    nx.draw(G,with_labels = True,node_color='b',node_size=500);

    from node2vec import Node2Vec
    # Generate walks
    node2vec = Node2Vec(G, dimensions=2, walk_length=20, num_walks=10,workers=4)
    # Learn embeddings 
    model = node2vec.fit(window=10, min_count=1)
    #model.wv.most_similar('1')
    model.wv.save_word2vec_format("embedding.emb") #save the embedding in file embedding.emb


    """
    Step 2: apply the clustering method. 
    Once you get vector representation of the nodes, you can cluster the nodes based on those representations. 
    See the example below.
    """

    from sklearn.cluster import KMeans
    import numpy as np


    X = np.loadtxt("embedding.emb", skiprows=1) # load the embedding of the nodes of the graph
    #print(X)
    # sort the embedding based on node index in the first column in X
    X=X[X[:,0].argsort()]; 
    #print(X)
    Z=X[0:X.shape[0],1:X.shape[1]]; # remove the node index from X and save in Z

    kmeans = KMeans(n_clusters=2, random_state=0).fit(Z) # apply kmeans on Z
    labels=kmeans.labels_  # get the cluster labels of the nodes.
    print(labels)
